In [19]:
from datasets import load_dataset, Dataset, load_from_disk
from interprot_sae import get_latents, load_models
from constants import linear_probe_dataset_path

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("SaProtHub/Dataset-GB1-fitness")["train"]

In [20]:
set(ds["stage"])

{'test', 'train', 'valid'}

In [21]:
ds.info
# indices = list(range(90)) + list(range(len(ds) - 10, len(ds)))
indices = list(range(100))
ds = Dataset.from_dict(ds[indices])

In [22]:
LAYER = 33
esm_mod, sae_mod = load_models(LAYER)

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def add_latent_batched(rows):
    seqs = rows["protein"]
    latents_SPZ = get_latents(seqs, LAYER, esm_mod, sae_mod)
    latents_SZ = latents_SPZ.mean(dim=1)
    rows["latents"] = latents_SZ
    return rows
new_ds = ds.map(add_latent_batched, batched=True)
new_ds.set_format("numpy", columns=["latents", "label"])


Map: 100%|██████████| 100/100 [00:00<00:00, 120.99 examples/s]


In [24]:
new_ds.save_to_disk("test.hf")

Saving the dataset (1/1 shards): 100%|██████████| 100/100 [00:00<00:00, 18889.01 examples/s]


In [25]:
new_ds = load_from_disk("test.hf")
new_ds

Dataset({
    features: ['protein', 'label', 'stage', 'latents'],
    num_rows: 100
})

In [31]:
new_ds["latents"][:].shape

(100, 4096)